# TP2

In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
#sc.stop()
sc = pyspark.SparkContext('local[*]')

In [3]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_trining.csv').drop('Unnamed: 0', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
#postulaciones.fillna(0,inplace=True)

In [5]:
#postulaciones['sepostulo']=1

In [6]:
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1112248724,NjlD,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1981-06-02,FEM
1,1112248724,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
2,1112304156,MV6eM8N,1.0,Full-time,Junior,Comunicacion,RH Talentum,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
3,1112445502,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC
4,1112254843,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1991-10-11,MASC


In [8]:
test = pd.read_csv("Sources/test_final_100k.csv")


In [9]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1976-03-29,FEM,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1987-06-27,MASC,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1982-02-17,FEM,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1949-12-24,MASC,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1986-05-27,FEM,1


In [10]:
def dates_format(date):
    print(date)
    try:
        date_new = date.split('-')
        date_new = int("".join(date_new))
        return date_new
    except AttributeError:
        return date

In [11]:
fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [12]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,1112248724,NjlD,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19810602.0,FEM
1,1112248724,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,MASC
2,1112304156,MV6eM8N,1.0,Full-time,Junior,Comunicacion,RH Talentum,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,MASC
3,1112445502,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,MASC
4,1112254843,MV6eM8N,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,MASC


In [13]:
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,3.0,0.0,0.0,0.0,0.0,0.0,3.0,19760329.0,FEM,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,3.0,0.0,0.0,3.0,0.0,19870627.0,MASC,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19820217.0,FEM,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,3.0,0.0,0.0,3.0,19491224.0,MASC,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,0.0,0.0,0.0,0.0,0.0,2.0,2.0,19860527.0,FEM,1


In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    postulaciones[col] = postulaciones.apply(lambda x: str(x[col]),axis=1)
    le.fit(postulaciones[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not test[col].dtype == 'O': continue
    test[col] = test.apply(lambda x: str(x[col]),axis=1)
    le.fit(test[col])
    test[col] = le.transform(test[col])
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo,sepostulo
0,0,739260,13661,1,1,29,337,3.0,0.0,0.0,0.0,0.0,0.0,3.0,19760329.0,0,1
1,1,739260,16942,1,1,29,337,0.0,0.0,3.0,0.0,0.0,3.0,0.0,19870627.0,1,1
2,2,739260,61148,1,1,29,337,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19820217.0,0,1
3,3,758580,2178,1,3,164,2725,0.0,0.0,0.0,3.0,0.0,0.0,3.0,19491224.0,1,1
4,4,758580,24726,1,3,164,2725,0.0,0.0,0.0,0.0,0.0,2.0,2.0,19860527.0,0,1


In [16]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop('sepostulo',axis=1,inplace=True)
test.drop('sepostulo',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [23]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [24]:
postulaciones.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1112248724,162302,1,4,119,1923,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19810602.0,1
1,1112248724,149565,1,4,119,1923,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
2,1112304156,149565,1,2,35,2574,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
3,1112445502,149565,1,4,35,1768,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2
4,1112254843,149565,1,4,63,123,0.0,0.0,0.0,0.0,0.0,3.0,2.0,19911011.0,2


In [25]:
test.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,739260,13661,1,1,29,337,3.0,0.0,0.0,0.0,0.0,0.0,3.0,19760329.0,0
1,739260,16942,1,1,29,337,0.0,0.0,3.0,0.0,0.0,3.0,0.0,19870627.0,1
2,739260,61148,1,1,29,337,0.0,0.0,0.0,0.0,0.0,0.0,3.0,19820217.0,0
3,758580,2178,1,3,164,2725,0.0,0.0,0.0,3.0,0.0,0.0,3.0,19491224.0,1
4,758580,24726,1,3,164,2725,0.0,0.0,0.0,0.0,0.0,2.0,2.0,19860527.0,0


In [ ]:
aviso = None
postulantes = None
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(data=postulaciones,label=resultado)
dtest = xgb.DMatrix(data=test, label=resultadob)
# specify parameters via map
param = {'max_depth':50, 'eta':1, 'silent':1, 'objective':'multi:softmax','num_class':3, 'subsample':1}
num_round = 50

bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [ ]:
preds[preds==0]